# A/B Testing Protocol

In [ ]:
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
#Our results from A/B test for click-throughs
A_views, A_clicks = 2745, 1958
B_views, B_clicks = 759, 523

In [ ]:
# Need to update the model in context
with pm.Model():
    
    # Define our guess on what the clicktrough might be. This is a ratio of success:failure (alpha:beta). 
    # It is ok for this to be uninformative (1:1) but is able to capture our assumptions.
    prior_A = pm.Beta('prior_A', alpha=3, beta=1)
    prior_B = pm.Beta('prior_B', alpha=3, beta=1)
    
    # Update the model with the observations from the experiment
    obs_A = pm.Binomial('like_A', n=A_views, p=prior_A, observed=A_clicks)
    obs_B = pm.Binomial('like_B', n=B_views, p=prior_B, observed=B_clicks)
    
    # Examine the lift by subtracting B from A (how much better/worse is B than A)
    pm.Deterministic('difference', prior_B - prior_A)
    pm.Deterministic('relation', (prior_B/prior_A)-1)
    
    # This starts the MCMC procedure for generating outcomes where we do draws of N.
    trace = pm.sample(draws=50000, start=pm.find_MAP(), progressbar=True)

In [ ]:
    
# We are intersted in the posterior probability, so want to discard the first bunch of drows.    
_ = pm.traceplot(trace[1000:])

# Plot the lift of B over A
_ = pm.plot_posterior(trace[1000:], varnames=['difference', 'relation'], ref_val=0, color='#87ceeb')

#pm.forestplot(trace)  #Uncomment to run a forest plot
summary = pm.summary(trace) # Discrete descriptions of the results

In [ ]:
#Joint Posterior
pA = trace[1000:]['prior_A']
pB = trace[1000:]['prior_B']
delta_posterior = trace[1000:]['difference']

# 3d plot
vals=np.vstack([pA, pB]).T
limit=0.8
df=pd.DataFrame(vals, columns=['A', 'B'])
df=df[df['A']<limit]
df=df[df['B']<limit]
g=sns.jointplot(x=df.A, y=df.B, kind='kde', n_levels=15)
g.ax_joint.plot([0.0, limit], [0.0, limit])